<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

// needed to generate the Table of contents 
// taken from github.com/kmahelona/ipython_notebook_goodies

<IPython.core.display.Javascript object>

(To work with Atom and Jupyter at the same time)

In [ ]:
%load_ext autoreload
%autoreload 2

# Tubofan Engine Predictive Maintenance (Template)
 
1. Problem Identification
2. **Data Wrangling**
   - Data Collection
       - Locating the data
       - Data loading
       - Data joining
    - Data Organization
        - File structure
        - Git & Github
    - Data Definition
        - Column names
        - Data types (numeric, categorical, timestamp, etc.)
        - Description of the columns
        - Count or percent per unique values or codes (including NA)
        - The range of values or codes
    - Data Cleaning
        - NA or missing data
        - Duplicates
3. Exploratory Data Analysis

4. Pre-processing and Training Data Development

5. Modeling

6. Documentation

# Data Wrangling

## Data Collection

In [ ]:
#load python packages
import os
import glob
import pandas as pd
#import datetime
#import seaborn as sns
#import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
#navigate to the data folder
os.chdir('..')
print(os.getcwd())

In [ ]:
# Download data in data/raw folder
! mkdir data/raw
! cd data/raw && wget -O turbofan.zip https://ti.arc.nasa.gov/c/6/ && unzip -o *.zip
! cd data/raw && rm *.zip

#NOTE the cd executed with the magic commanda do not change the working directory of the python script
#We are still working off the project root folder

In [ ]:
# define column names
sensors_list = ["s{}".format(s) for s in range(1,22)]
train_cols = ['unit_number','cycle_time','op_setting_1', 'op_setting_2', 'op_setting_3'] + sensors_list
test_cols = train_cols
RUL_cols = ['RUL']


In [ ]:
# load Training data
file_paths = glob.glob("data/raw/train_*.txt")

df_list = []
for file_path in file_paths : 
    # read txt file
    individual_df = pd.read_csv(file_path, sep=' ', header=None, usecols = [i for i in range(26)])
    individual_df.columns = train_cols
    #extract dataset Id from filename
    data_set=file_path[-9:-4]
    individual_df['dataset']=file_path[-9:-4] 
    #append temporary dataframe to list
    df_list.append(individual_df)
#merge into single dataframe
df_train=pd.concat(df_list)
df_train.head()

# write to csv
df_train.to_csv('data/train.csv', index=False)

In [ ]:
# load Test data
file_paths = glob.glob("data/raw/test_*.txt")

df_list = []
for file_path in file_paths : 
    # read txt file
    individual_df = pd.read_csv(file_path, sep=' ', header=None, usecols = [i for i in range(26)])
    individual_df.columns = test_cols
    #extract dataset Id from filename
    data_set=file_path[-9:-4]
    individual_df['dataset']=file_path[-9:-4] 
    #append temporary dataframe to list
    df_list.append(individual_df)
#merge into single dataframe
df_test=pd.concat(df_list)
df_test.head()

# write to csv
df_test.to_csv('data/test.csv', index=False)

In [ ]:
# load RUL data
file_paths = glob.glob("data/raw/RUL_*.txt")

df_list = []
for file_path in file_paths : 
    # read txt file
    individual_df = pd.read_csv(file_path, sep=' ', header=None, usecols=[0])
    individual_df.columns = RUL_cols
    #extract dataset Id from filename
    data_set=file_path[-9:-4]
    individual_df['dataset']=file_path[-9:-4] 
    #append temporary dataframe to list
    df_list.append(individual_df)
#merge into single dataframe
df_RUL=pd.concat(df_list)
df_RUL.head()

# write to csv
df_RUL.to_csv('data/RUL.csv', index=False)

Include description of filestructure